For this project we aim to use the EC2 Instance Connect End Point to connect to resources in the VPC (Virtual Private Cloud) using the AWS CLI. Once set up an IAM User will be created with programmatic access which will be configured to our computer. This will allow us to use the EC2 Instance connect endpoint to connect to resources in the private subnet and the public subnet of our VPC using AWS CLI

In [ ]:
Requirements: 
- homebrew 
- iterm2
- AWS CLI

AWS - IAMS 
(Creating new Administrative Access User)

We're going to create an administrative Access User Using the following steps: 
1. Create User

<img src="Images/UsersSelect.png" alt="Drawing" style="width: 300px;"/>

![CreateUser](Images/CreateUser.png)

2. Once created Attach required policies: 
![UserName](Images/UserName.png)
![AttachPolicies](Images/AttachPolicies.png)

Creating Secret Access Key for the IAM User 
![ClickUser](Images/ClickUser.png)
![CreateAccessKey](Images/CreateAccessKey.png)
![CLIAttribute](Images/CLIAttribute.png)
![AccessKeyShow](Images/AccessKeyShow.png)
